In [11]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pyaudio

%matplotlib tk

# 设置音频参数
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 48000
CHUNK = 4800


# 设置要分析的四个特定频率
target_frequencies = [697, 770, 852, 941, 1209, 1336, 1477, 1633, 1958]
frequency_range = 20  # 频率范围

# 计算频率范围的边界
frequency_bounds = [(f - frequency_range/2, f + frequency_range/2) for f in target_frequencies]
# 创建用于绘图的图表和子图
fig, ax = plt.subplots()
bars = ax.bar(range(len(target_frequencies)), np.zeros(len(target_frequencies)))

# 设置y轴范围
ax.set_ylim(4,7)  # 替换为适当的值

# 设置x轴刻度和标签
ax.set_xticks(range(len(target_frequencies)))
ax.set_xticklabels([f"{freq} Hz" for freq in target_frequencies])

trigger = 0

# 初始化音频输入
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

def decode(input):
    global trigger
    trig_signal = 0
    if 8 in input:
        trig_signal = 1
    if trigger != trig_signal:
        print(input)
    
    
# 更新图表的回调函数
def update(frame):
    try:
        # 读取音频数据
        data = np.frombuffer(stream.read(CHUNK), dtype=np.int16)
        
        fft_result = np.fft.fft(data)
        frequencies_fft = np.fft.fftfreq(len(fft_result), 1/RATE)
        
        # 计算特定频率分量的幅度，扩大了频率范围
        magnitudes = [np.mean(np.abs(fft_result[(frequencies_fft >= f_low) & (frequencies_fft <= f_high)]))
                  for (f_low, f_high) in frequency_bounds]
        
        result = []
        for i in range(len(target_frequencies)): 
            bar = bars[i]
            magnitude = magnitudes[i]
            magnitude_log = np.log10(magnitude + 1)  # 加1是为了避免log(0)
            bar.set_height(magnitude_log)
            if magnitude_log > 5:
                result.append(i)
        if len(result)!=0:
            decode(result)
        
        return bars
    except Exception as e:
        print(f"Error: {e}")

# 设置动画更新间隔
ani = animation.FuncAnimation(fig, update, blit=False, interval=10)

# 显示图表
plt.show(block=True)


# 关闭音频流和PyAudio对象
stream.stop_stream()
stream.close()
p.terminate()


C:\Users\Tom_yang\AppData\Local\Temp\ipykernel_13580\3463576577.py:80: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  ani = animation.FuncAnimation(fig, update, blit=False, interval=10)


[0, 1, 2, 3, 4, 5, 8]
[3, 7, 8]
[0, 2, 3, 4, 6, 7, 8]
[0, 1, 2, 3, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 8]
[0, 1, 2, 3, 7, 8]
[1, 8]
[0, 1, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[4, 8]
